In [1]:
# !wget https://raw.githubusercontent.com/mesolitica/malaysian-dataset/master/llm-benchmark/BM-pt3/BM-A-pt3

In [2]:
from tqdm import tqdm
import os
import random

In [3]:
apikey = ''

In [4]:
import requests

def curl(value):
    headers = {
        'Content-Type': 'application/json',
        'api-key': apikey,
    }

    params = {
        'api-version': '2023-03-15-preview',
    }

    json_data = {
        'messages': [
            {'role': 'system', 'content':'answer only, no need to explain'},
            {
                'role': 'user',
                'content': value,
            },
        ],
        'max_tokens': 100,
        'temperature': 0,
    }

    response = requests.post(
        'https://nous.openai.azure.com/openai/deployments/nous/chat/completions',
        params=params,
        headers=headers,
        json=json_data,
        timeout = 120,
    )
    
    return response

In [5]:
with open('BM-A-pt3') as fopen:
    text = fopen.read()
    
questions = []
for t in text.split('no: ')[1:]:
    t = t.strip()
    no = t.split('\n')[0]
    objektif = t.split('objektif: ')[1].split('\n')[0]
    soalan = t.split('soalan:')[1].split('jawapan:')[0].strip()
    jawapan = t.split('jawapan: ')[1].split(',')[0].strip()
    data = {
        'no': no,
        'objektif': objektif,
        'soalan': soalan,
        'jawapan': jawapan,
    }
    questions.append(data)

In [6]:
arange = set(range(len(questions)))

In [7]:
def convert_prompt(row, answer = False):
    if answer:
        prompt = f"""
objektif: {row['objektif']}
soalan: {row['soalan']}
jawapan: {row['jawapan']}
    """
    else:
        prompt = f"""
objektif: {row['objektif']}
soalan: {row['soalan']}
    """
    return prompt.strip()

In [8]:
i = 0
shots = random.sample(arange - {i}, 3)
prompts = []
for no, s in enumerate(shots):
    prompts.append(f'Contoh soalan {no + 1}\n' + convert_prompt(questions[s], answer = True))

prompts.append(convert_prompt(questions[i]))
prompt = '\n\n'.join(prompts)
print(prompt)

Contoh soalan 1
objektif: Pilih peribahasa atau maksud peribahasa yang paling sesuai.
soalan: Peribahasa cubit peha kanan, peha kiri juga terasa membawa maksud
A. sesuatu pekerjaan yang sukar hendaklah dilakukan dengan bersungguh-sungguh
B. jika ahli keluarga mendapat susah, yang lain akan turut susah
C. bertindak dengan tergesa-gesa, akhirnya akan merugikan diri sendiri.
D. orang yang malu bertanya, akan tersesat di pertengahan jalan
jawapan: B

Contoh soalan 2
objektif: Bahagian yang di dalam kurungan dalam ayat-ayat yang berikut mungkin mengandungi kesalahan bahasa dan mungkin juga tidak. Tandakan sama ada A, B atau C jika ayat itu mengandungi kesalahan dan tandakan D jika tiada kesalahan.
soalan: Halimah percaya (percaya benar) anaknya tidak terbabit dengan tuduhan jenayah kolar putih.
A. sungguh percaya benar
B. teramat percaya benar
C. sangat percaya benar
D. percaya benar
jawapan: D

Contoh soalan 3
objektif: Lengkapkan ayat-ayat yang berikut dengan memilih jawapan yang paling s

In [9]:
r = curl(prompt)

In [10]:
r.json()['choices'][0]['message']['content']

'jawapan: D'

In [11]:
for i in tqdm(range(len(questions))):
    shots = random.sample(arange - {i}, 1)
    prompts = []
    for no, s in enumerate(shots):
        prompts.append(f'Contoh soalan {no + 1}\n' + convert_prompt(questions[s], answer = True))

    prompts.append(convert_prompt(questions[i]))
    prompt = '\n\n'.join(prompts)
    try:
        r = curl(prompt)
        answer = r.json()['choices'][0]['message']['content']
        questions[i]['output'] = answer
    except:
        pass

100%|███████████████████████████████████████████| 54/54 [01:08<00:00,  1.26s/it]


In [12]:
import json

with open('output-3shot.json', 'w') as fopen:
    json.dump(questions, fopen)

In [13]:
filtered = [q for q in questions if 'output' in q]
len(filtered)

53

In [15]:
correct = 0
for q in filtered:
    correct += q['output'].split('.')[0] == q['jawapan']

In [16]:
(correct / len(filtered)) * 100

24.528301886792452